In [1]:
import os
import time

import bs4
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns

from collections import defaultdict
# import jupyterthemes as jt 
# from jupyterthemes.stylefx import set_nb_theme
# from jupyterthemes import get_themes 

%matplotlib inline
pd.options.display.max_columns = 99
os.chdir("/Users/yenchenchou/Documents/RMDS_YC/RiskScore/RMDS_COVID19_riskgenerator")

In [3]:
df = pd.read_csv(
    "data/Core-USA-July2020-Release-CORE_POI-2020_06-2020-07-13/core_poi-part1.csv.gz",
    compression = "gzip")

In [4]:
df.head()

,safegraph_place_id,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,iso_country_code,phone_number,open_hours,category_tags,opened_on,closed_on,tracking_opened_since,tracking_closed_since
0,sg:0026d587c9b145b58272cbac7bdea56a,NaN,Vallone A John DDS,NaN,NaN,Offices of Dentists,Offices of Dentists,621210.0,41.158062,-87.891611,35 Briarcliff Prof Ctr,Bourbonnais,IL,60914,US,1.815939e+10,NaN,NaN,NaN,NaN,NaN,2019-07
1,sg:0037c4dc85f642c6aeaaf318e41b1bf7,NaN,Mitchell Cosmetology College,NaN,NaN,"Colleges, Universities, and Professional Schools","Colleges, Universities, and Professional Schools",611310.0,33.243518,-86.816563,116 1st St S,Alabaster,AL,35007,US,1.205664e+10,"{ ""Mon"": [], ""Tue"": [[""12:00"", ""16:00""]], ""Wed...",NaN,NaN,NaN,NaN,2019-07
2,sg:0041bdf50f984826acd68ce19922ecd4,NaN,Sullivan Chris M Dr,NaN,NaN,"Other Professional, Scientific, and Technical ...",Veterinary Services,541940.0,32.153705,-91.715387,403 Eighth St,Winnsboro,LA,71295,US,NaN,NaN,NaN,NaN,NaN,NaN,2019-07
3,sg:005761d9681947629bf02a64cfd4aec2,NaN,U Pick Carlsbad Strawberry Co,NaN,NaN,Specialty Food Stores,Fruit and Vegetable Markets,445230.0,33.135632,-117.326437,1000 Cannon Rd,Carlsbad,CA,92008,US,NaN,"{ ""Mon"": [[""9:00"", ""18:00""]], ""Tue"": [[""9:00"",...",NaN,NaN,NaN,NaN,2019-07
4,sg:007236f9d77d4158bbd4d55f39d98e49,NaN,CJ's Auto Glass Repair,NaN,NaN,Lumber and Other Construction Materials Mercha...,Other Construction Material Merchant Wholesalers,423390.0,40.470182,-74.408106,190 State Route 18,East Brunswick,NJ,8816,US,1.732994e+10,"{ ""Mon"": [[""8:00"", ""20:00""]], ""Tue"": [[""8:00"",...",NaN,NaN,NaN,NaN,2019-07


In [5]:
df.tail()

,safegraph_place_id,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,iso_country_code,phone_number,open_hours,category_tags,opened_on,closed_on,tracking_opened_since,tracking_closed_since
1179090,sg:ff9c737df182446499c91a08773f23df,NaN,Big Ed's,NaN,NaN,Wired and Wireless Telecommunications Carriers,Wireless Telecommunications Carriers (except S...,517312.0,42.680827,-89.028648,410 W Miles Rd,Janesville,WI,53545,US,1.608753e+10,"{ ""Mon"": [[""9:00"", ""17:00""]], ""Tue"": [[""9:00"",...",NaN,NaN,NaN,NaN,2019-07
1179091,sg:ffa766b1e7f94868933e13e67eeb0d30,NaN,Roadrunner Markets,SG_BRAND_37d81d07757147da,Roadrunner Markets,Grocery Stores,Supermarkets and Other Grocery (except Conveni...,445110.0,35.528562,-82.998621,1980 Dellwood Rd,Waynesville,NC,28786,US,NaN,NaN,NaN,NaN,NaN,2019-07,2019-07
1179092,sg:ffa7af0f41b74d8887c27b53e2ecba4c,NaN,BP,SG_BRAND_5cfdb867e96374c7883b31d6928cc4cb,BP,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,39.762534,-86.378627,8133 E US Highway 36,Avon,IN,46123,US,NaN,"{ ""Mon"": [[""0:00"", ""23:59""]], ""Tue"": [[""0:00"",...",NaN,NaN,NaN,NaN,2019-07
1179093,sg:ffc876aee3464f13ad9dfcee40416a8d,NaN,Wallhanger Taxidermy,NaN,NaN,Shoe Stores,Shoe Stores,448210.0,34.348581,-98.307263,220 E Wyoming St,Walters,OK,73572,US,1.580699e+10,"{ ""Mon"": [[""8:00"", ""20:00""]], ""Tue"": [[""8:00"",...",NaN,NaN,NaN,NaN,2019-07
1179094,sg:ffdfe2a321d047cba9721f30871019f3,NaN,CODI Educational Services,NaN,NaN,"Colleges, Universities, and Professional Schools","Colleges, Universities, and Professional Schools",611310.0,33.704860,-84.772479,3713 Kings Hwy Ste 1004,Douglasville,GA,30135,US,1.678403e+10,NaN,NaN,NaN,NaN,NaN,2019-07


In [6]:
df_5 = pd.read_csv(
    "data/Core-USA-July2020-Release-CORE_POI-2020_06-2020-07-13/core_poi-part5.csv.gz",
    compression = "gzip")

In [7]:
df_5.head()

,safegraph_place_id,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,iso_country_code,phone_number,open_hours,category_tags,opened_on,closed_on,tracking_opened_since,tracking_closed_since
0,sg:000e24b5d8494098aca6c6afaaf70f60,NaN,Surgoinsville Elementary,NaN,NaN,Elementary and Secondary Schools,Elementary and Secondary Schools,611110.0,36.461528,-82.867835,1010 Main St,Surgoinsville,TN,37873,US,1.423345e+10,NaN,NaN,NaN,NaN,NaN,2019-07
1,sg:0032253ee1a24b0099aad73565b084c8,sg:a7955000f2ef4f7b9005ca3d3460782a,Dental Crown in an Hour,NaN,NaN,Offices of Dentists,Offices of Dentists,621210.0,26.494321,-81.849833,16960 Alico Mission Way Ste 100 101,Fort Myers,FL,33908,US,1.239322e+10,"{ ""Mon"": [[""9:00"", ""17:00""]], ""Tue"": [[""9:00"",...",NaN,NaN,NaN,NaN,2019-07
2,sg:003267592fd54e2d9137cb68056cd092,NaN,Wako Japanese Restaurant,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,37.782876,-122.461561,211 Clement St,San Francisco,CA,94118,US,1.415682e+10,"{ ""Mon"": [], ""Tue"": [[""17:30"", ""22:00""]], ""Wed...","Japanese Food,Sushi,Fine Dining",NaN,NaN,NaN,2019-07
3,sg:0037771296084487a7eaf3730a4ad221,NaN,Jewel-Osco,SG_BRAND_5f0f84f03b60ceb7b549163ce91da6f4,Jewel-Osco,Grocery Stores,Supermarkets and Other Grocery (except Conveni...,445110.0,42.015745,-87.839285,481 Busse Hwy,Park Ridge,IL,60068,US,NaN,"{ ""Mon"": [[""6:00"", ""24:00""]], ""Tue"": [[""6:00"",...",NaN,NaN,NaN,NaN,2019-07
4,sg:0040ef48637a4502bc8fefa55b0c3459,NaN,Kuakini Medical Center,NaN,NaN,General Medical and Surgical Hospitals,General Medical and Surgical Hospitals,622110.0,21.321950,-157.857204,347 N Kuakini St,Honolulu,HI,96817,US,NaN,NaN,NaN,NaN,NaN,NaN,2019-07


In [8]:
df_5.tail()

,safegraph_place_id,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,iso_country_code,phone_number,open_hours,category_tags,opened_on,closed_on,tracking_opened_since,tracking_closed_since
1117132,sg:ff5c2682029940e9b2276e18c24f5089,NaN,Wolf's Bar B Q,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,38.039164,-87.573914,6600 N 1st Ave,Evansville,IN,47710,US,1.812425e+10,"{ ""Mon"": [], ""Tue"": [[""11:00"", ""21:00""]], ""Wed...",BBQ and Southern Food,NaN,NaN,NaN,2019-07
1117133,sg:ffa10209c40c4b808ce2357fd77b2e97,NaN,American Insurance Agencies of Texas,NaN,NaN,"Agencies, Brokerages, and Other Insurance Rela...",Insurance Agencies and Brokerages,524210.0,32.793644,-96.616839,1331 US Highway 80 E,Mesquite,TX,75150,US,1.972290e+10,"{ ""Mon"": [[""9:00"", ""17:00""]], ""Tue"": [[""9:00"",...",NaN,NaN,NaN,NaN,2019-07
1117134,sg:ffa963a61db648549bc5678ec35ab19c,NaN,Columbia Memorial Hospial Urgent Care Center,NaN,NaN,Outpatient Care Centers,Freestanding Ambulatory Surgical and Emergency...,621493.0,42.419429,-73.685941,2827 Rt 9,Valatie,NY,12184,US,NaN,NaN,NaN,NaN,NaN,NaN,2019-07
1117135,sg:ffd5c79931b84b7a9814be3e7661ec0b,NaN,Sun Surf Veterinary Hospital,NaN,NaN,"Other Professional, Scientific, and Technical ...",Veterinary Services,541940.0,33.721364,-118.074965,16571 Pacific Coast Hwy,Sunset Beach,CA,90742,US,1.562296e+10,"{ ""Mon"": [[""8:00"", ""18:00""]], ""Tue"": [[""8:00"",...",NaN,NaN,NaN,NaN,2019-07
1117136,sg:ffe7c59404be4e74b34b23208056d574,NaN,Tool A Rama,NaN,NaN,Building Material and Supplies Dealers,Hardware Stores,444130.0,36.774187,-90.419461,1880 N Westwood Blvd,Poplar Bluff,MO,63901,US,1.573778e+10,NaN,NaN,NaN,2020-02,NaN,2019-07
